# Working notebook

In [ ]:
import pickle

result = pickle.load(open("result.pkl", "rb"))

In [ ]:
import mesa
from model import BuildingModel
import numpy as np

ITERATIONS = 10
MAX_STEPS = 10000

params = {    
    "N": range(1000, 20000, 2000 ), 
    "perc_uninformed_agents" : 0,
    "alpha" : 1,
    "beta" : 0.5,
    "speed_mean" : np.arange(0.1, 1, 0.2),
    "speed_variance" : 0.4,
}


result = mesa.batch_run(
    BuildingModel,
    parameters=params,
    iterations=ITERATIONS, 
    max_steps=MAX_STEPS,
    number_processes=12,
    data_collection_period=1,
    display_progress=True,
)

In [ ]:
import pandas as pd
result_df = pd.DataFrame(result)

result_df

In [ ]:
result_df.info()

In [ ]:
import matplotlib.pyplot as plt

def get_agents_count(df):
    
    df1 = df
    
    agents_count_mean = np.zeros(shape= (4, MAX_STEPS + 1) )

    for i in df1['RunId'].unique():
        df2 = df1[df1.RunId == i][["Step", "Agents_proportion"]].sort_values(by="Step")

        for j in range(len(df2)):
            agents_count_mean[0][j] += dict(df['Agents_proportion'].iloc[j])['slow']
            agents_count_mean[1][j] += dict(df['Agents_proportion'].iloc[j])['medium']
            agents_count_mean[2][j] += dict(df['Agents_proportion'].iloc[j])['fast']
            agents_count_mean[3][j] += dict(df['Agents_proportion'].iloc[j])['uninformed']


    agents_count_mean /= ITERATIONS

    return agents_count_mean

agent_types = ["slow", "medium", "fast", "uninformed"]

for i, agent_type in zip(get_agents_count(result_df), agent_types):
    plt.plot(i, label=agent_type)

plt.xlabel("Steps")
plt.ylabel("People inside")
plt.legend(title="Agent type")


In [ ]:
def get_active_agents_mean(df, N, speed_mean):
    
    df1 = df[(df.N == N) & (np.abs(df.speed_mean - speed_mean) < 1e-4)]
    
    active_agents_mean = np.zeros(MAX_STEPS + 1)

    for i in df1['RunId'].unique():
        df2 = df1[df1.RunId == i][["Step", "Active_agents"]].sort_values(by="Step")
        active_agents = np.array (df2["Active_agents"].tolist() )
        active_agents_mean += active_agents

    active_agents_mean /= ITERATIONS

    return active_agents_mean



In [ ]:
import matplotlib.pyplot as plt

speed_mean = 0.9

for i in params["N"]:
    r = get_active_agents_mean(result_df, i, speed_mean)
    plt.plot(r, label = str(i)[:6])

plt.xlabel("Steps")
plt.ylabel("People inside")
# plt.title("Number of people inside the building with different \n variances and mean fixed to 0.8 (Gaussian distribution),\n averaged over 10 runs")

plt.legend(title="Starting people")
plt.show()
